# 🧠 数学推理 SFT 数据构造流水线 (Evol-Instruct + PoT)
本 Notebook 整合了从 **种子数据采样**、**指令进化(Evol)**、**程序解题生成(PoT)** 到 **沙箱执行验证** 的完整闭环。

### 1. 环境准备与依赖安装

In [ ]:
pip install requests tqdm

### 2. 第一阶段：种子数据采样
从原始数据集（如 GSM8K）中抽取基础样本作为“种子”。

In [ ]:
import json
import random
import os

INPUT_FILE = "../data/gsm8k_train.jsonl"
SEED_FILE = "../data/seed_samples.jsonl"
SAMPLE_SIZE = 10

os.makedirs("../data", exist_ok=True)

def sample_seeds():
    if not os.path.exists(INPUT_FILE):
        print(f"❌ 找不到输入文件 {INPUT_FILE}，请确保路径正确。")
        return
    
    with open(INPUT_FILE, 'r', encoding='utf-8') as f:
        data = [json.loads(line) for line in f if line.strip()]
    
    sampled = random.sample(data, min(len(data), SAMPLE_SIZE))
    
    with open(SEED_FILE, 'w', encoding='utf-8') as f:
        for entry in sampled:
            seed = {
                "id": random.randint(1000, 9999),
                "seed_question": entry['question'],
                "original_answer": entry['answer']
            }
            f.write(json.dumps(seed, ensure_ascii=False) + '\n')
    print(f"✅ 已采样 {len(sampled)} 条种子数据至 {SEED_FILE}")

sample_seeds()

### 3. 第二阶段：指令进化与 PoT 解题生成
利用大模型（DeepSeek-V3）将简单问题复杂化，并生成 Python 代码解决方案。

In [ ]:
import requests
import time
from tqdm.notebook import tqdm

API_KEY = "sk-lrdpxzsnhsbckhjrzekbrtccomruhcwzyrlwbroqwojtwtsw"
BASE_URL = "https://api.siliconflow.cn/v1/chat/completions"
MODEL_NAME = "deepseek-ai/DeepSeek-V3"
EVOL_OUTPUT = "../data/evolved_samples.jsonl"

def call_llm(prompt):
    headers = {"Authorization": f"Bearer {API_KEY}", "Content-Type": "application/json"}
    payload = {
        "model": MODEL_NAME,
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.7,
        "max_tokens": 2048
    }
    try:
        res = requests.post(BASE_URL, json=payload, headers=headers, timeout=120)
        return res.json()['choices'][0]['message']['content']
    except: return None

def process_evol():
    with open(SEED_FILE, 'r') as f:
        seeds = [json.loads(l) for l in f]
    
    for entry in tqdm(seeds, desc="进化与解题"):
        # 1. 指令进化
        evol_p = f"请将此数学题重写得更复杂场景化，只输出问题：{entry['seed_question']}"
        evolved_q = call_llm(evol_p)
        if not evolved_q: continue
        
        # 2. PoT 生成
        pot_p = f"编写 solve() 函数解决此问题并返回数值：{evolved_q}"
        solution = call_llm(pot_p)
        if not solution: continue
        
        res = {"id": entry['id'], "evolved_question": evolved_q, "pot_solution": solution}
        with open(EVOL_OUTPUT, 'a', encoding='utf-8') as f:
            f.write(json.dumps(res, ensure_ascii=False) + '\n')

process_evol()

### 4. 第三阶段：沙箱代码验证
自动提取生成的 Python 代码并在隔离环境中运行，确保生成的解法是正确且可执行的。

In [ ]:
import re
import subprocess

FINAL_OUTPUT = "../data/verified_textbook.jsonl"

def extract_code(text):
    match = re.search(r"```python\s*(.*?)\s*```", text, re.DOTALL)
    return match.group(1) if match else None

def run_code(code):
    try:
        result = subprocess.run(['python3', '-c', code], capture_output=True, text=True, timeout=5)
        return (True, result.stdout.strip()) if result.returncode == 0 else (False, result.stderr)
    except: return (False, "Timeout")

def verify_pipeline():
    valid_data = []
    with open(EVOL_OUTPUT, 'r') as f:
        for line in f:
            entry = json.loads(line)
            code = extract_code(entry['pot_solution'])
            if not code: continue
            
            success, output = run_code(code)
            if success and output:
                valid_data.append({
                    "instruction": entry['evolved_question'],
                    "output": entry['pot_solution'],
                    "answer": output
                })
    
    with open(FINAL_OUTPUT, 'w', encoding='utf-8') as f:
        for item in valid_data:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')
    print(f"✅ 验证完成！共有 {len(valid_data)} 条高质量样本存入 {FINAL_OUTPUT}")

verify_pipeline()